In [1]:
cd ..

/Users/wliao0504/Library/Mobile Documents/com~apple~CloudDocs/Repo/hca-lab/CLIF-MIMIC


/Users/wliao0504/.pyenv/versions/3.13.0/lib/python3.13/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# src/tables/patient.py
import numpy as np
import pandas as pd
import duckdb
import logging
from importlib import reload
import src.utils
reload(src.utils)
from src.utils import construct_mapper_dict, load_mapping_csv, \
    rename_and_reorder_cols, save_to_rclif, setup_logging, mimic_table_pathfinder

loaded configuration from config.json
loaded configuration from config.json


In [3]:
setup_logging()

PATIENT_COL_NAMES = [
    "patient_id", "race_name", "race_category", "ethnicity_name", "ethnicity_category",
    "sex_name", "sex_category", "birth_date", "death_dttm", "language_name", "language_category"
]

2025-01-05 21:57:34,816 - INFO - logging initialized and saved at logs/test.log


In [4]:
race_ethnicity_mapping = load_mapping_csv("race_ethnicity")
race_mapper_dict = construct_mapper_dict(race_ethnicity_mapping, "mimic_race", "race")
ethnicity_mapper_dict = construct_mapper_dict(race_ethnicity_mapping, "mimic_race", "ethnicity")

# load mimic data
mimic_patients = pd.read_parquet(mimic_table_pathfinder("patients"))
mimic_admissions = pd.read_parquet(mimic_table_pathfinder("admissions"))

In [11]:
query = """
SELECT 
    subject_id as patient_id, 
    hadm_id as hospitalization_id,
    race as race_name, 
    race as ethnicity_name,
    admittime as admittime
FROM mimic_admissions
"""
race_ethn = duckdb.query(query).df()
race_ethn["race_category"] = race_ethn["race_name"].map(race_mapper_dict)
race_ethn["ethnicity_category"] = race_ethn["ethnicity_name"].map(ethnicity_mapper_dict)
query = """
SELECT 
    patient_id,
    hospitalization_id,
    race_name,
    race_category,
    ethnicity_name,
    ethnicity_category,
    admittime,
    CASE
        WHEN (race_category IN ('Other', 'Unknown')) AND (ethnicity_category IN ('Other', 'Unknown')) THEN 1
        ELSE 0
    END AS true_noninfo
FROM race_ethn
"""
race_ethn = duckdb.query(query).df()
race_ethn

,patient_id,hospitalization_id,race_name,race_category,ethnicity_name,ethnicity_category,admittime,true_noninfo
0,10000032,22595853,WHITE,White,WHITE,Non-Hispanic,2180-05-06 22:23:00,0
1,10000032,22841357,WHITE,White,WHITE,Non-Hispanic,2180-06-26 18:27:00,0
2,10000032,25742920,WHITE,White,WHITE,Non-Hispanic,2180-08-05 23:44:00,0
3,10000032,29079034,WHITE,White,WHITE,Non-Hispanic,2180-07-23 12:35:00,0
4,10000068,25022803,WHITE,White,WHITE,Non-Hispanic,2160-03-03 23:16:00,0
...,...,...,...,...,...,...,...,...
546023,19999828,25744818,WHITE,White,WHITE,Non-Hispanic,2149-01-08 16:44:00,0
546024,19999828,29734428,WHITE,White,WHITE,Non-Hispanic,2147-07-18 16:23:00,0
546025,19999840,21033226,WHITE,White,WHITE,Non-Hispanic,2164-09-10 13:47:00,0
546026,19999840,26071774,WHITE,White,WHITE,Non-Hispanic,2164-07-25 00:27:00,0


In [27]:
test_patient_ids = [10005236, 10016673, 10017492, 10006513, 10304619, 10617011, 10360391, 10375831,11066451]
race_ethn_test = race_ethn[race_ethn["patient_id"].isin(test_patient_ids)]
race_ethn_test

,patient_id,hospitalization_id,race_name,race_category,ethnicity_name,ethnicity_category,admittime,true_noninfo
324,10005236,23384508,WHITE - OTHER EUROPEAN,White,WHITE - OTHER EUROPEAN,Non-Hispanic,2180-06-26 20:22:00,0
325,10005236,25656545,UNKNOWN,Unknown,UNKNOWN,Unknown,2177-11-29 11:30:00,1
412,10006513,28504108,HISPANIC/LATINO - DOMINICAN,Other,HISPANIC/LATINO - DOMINICAN,Hispanic,2125-05-06 11:22:00,0
413,10006513,29846618,OTHER,Other,OTHER,Unknown,2127-03-27 14:52:00,1
955,10016673,29103261,HISPANIC/LATINO - MEXICAN,Other,HISPANIC/LATINO - MEXICAN,Hispanic,2188-03-18 18:00:00,0
985,10017492,27417763,PATIENT DECLINED TO ANSWER,Unknown,PATIENT DECLINED TO ANSWER,Unknown,2116-06-26 18:25:00,1
986,10017492,27672872,PATIENT DECLINED TO ANSWER,Unknown,PATIENT DECLINED TO ANSWER,Unknown,2114-03-19 20:05:00,1
15902,10304619,21512114,HISPANIC OR LATINO,Other,HISPANIC OR LATINO,Hispanic,2161-06-19 00:47:00,0
15903,10304619,23891877,HISPANIC/LATINO - PUERTO RICAN,Other,HISPANIC/LATINO - PUERTO RICAN,Hispanic,2171-01-19 01:04:00,0
15904,10304619,24179237,HISPANIC/LATINO - PUERTO RICAN,Other,HISPANIC/LATINO - PUERTO RICAN,Hispanic,2174-06-17 06:37:00,0


In [33]:
query = """
SELECT 
    patient_id, 
    race_name,
    race_category,
    ethnicity_name,
    ethnicity_category,
    COUNT(*) AS count,
    MAX(admittime) AS most_recent,
    true_noninfo,
    ROW_NUMBER() OVER (
        PARTITION BY patient_id 
        ORDER BY 
            count DESC, 
            true_noninfo,
            most_recent DESC
            ) 
        AS rn
FROM race_ethn_test
GROUP BY patient_id, race_name, race_category, ethnicity_name, ethnicity_category, true_noninfo
"""
race_ethn_ranked_test = duckdb.query(query).df()
race_ethn_ranked_test

,patient_id,race_name,race_category,ethnicity_name,ethnicity_category,count,most_recent,true_noninfo,rn
0,10006513,HISPANIC/LATINO - DOMINICAN,Other,HISPANIC/LATINO - DOMINICAN,Hispanic,1,2125-05-06 11:22:00,0,1
1,10006513,OTHER,Other,OTHER,Unknown,1,2127-03-27 14:52:00,1,2
2,10304619,HISPANIC/LATINO - PUERTO RICAN,Other,HISPANIC/LATINO - PUERTO RICAN,Hispanic,7,2174-06-17 06:37:00,0,1
3,10304619,HISPANIC OR LATINO,Other,HISPANIC OR LATINO,Hispanic,1,2161-06-19 00:47:00,0,2
4,10617011,HISPANIC/LATINO - PUERTO RICAN,Other,HISPANIC/LATINO - PUERTO RICAN,Hispanic,1,2157-06-14 21:33:00,0,1
5,10617011,HISPANIC OR LATINO,Other,HISPANIC OR LATINO,Hispanic,1,2154-05-01 03:59:00,0,2
6,10360391,BLACK/AFRICAN AMERICAN,Black or African American,BLACK/AFRICAN AMERICAN,Non-Hispanic,3,2204-04-08 06:08:00,0,1
7,10360391,WHITE,White,WHITE,Non-Hispanic,1,2194-04-20 07:15:00,0,2
8,10375831,ASIAN - ASIAN INDIAN,Asian,ASIAN - ASIAN INDIAN,Non-Hispanic,3,2134-07-02 17:48:00,0,1
9,10375831,OTHER,Other,OTHER,Unknown,1,2130-01-01 16:58:00,1,2


In [37]:
query_new = """
SELECT 
    patient_id,
    race_name,
    race_category,
    ethnicity_name,
    ethnicity_category,
    rn
FROM race_ethn_ranked_test
WHERE rn = 1
"""
df2 = duckdb.query(query_new).df()
df2

,patient_id,race_name,race_category,ethnicity_name,ethnicity_category,rn
0,10006513,HISPANIC/LATINO - DOMINICAN,Other,HISPANIC/LATINO - DOMINICAN,Hispanic,1
1,10304619,HISPANIC/LATINO - PUERTO RICAN,Other,HISPANIC/LATINO - PUERTO RICAN,Hispanic,1
2,10617011,HISPANIC/LATINO - PUERTO RICAN,Other,HISPANIC/LATINO - PUERTO RICAN,Hispanic,1
3,10360391,BLACK/AFRICAN AMERICAN,Black or African American,BLACK/AFRICAN AMERICAN,Non-Hispanic,1
4,10375831,ASIAN - ASIAN INDIAN,Asian,ASIAN - ASIAN INDIAN,Non-Hispanic,1
5,11066451,WHITE,White,WHITE,Non-Hispanic,1
6,10005236,WHITE - OTHER EUROPEAN,White,WHITE - OTHER EUROPEAN,Non-Hispanic,1
7,10016673,HISPANIC/LATINO - MEXICAN,Other,HISPANIC/LATINO - MEXICAN,Hispanic,1
8,10017492,PATIENT DECLINED TO ANSWER,Unknown,PATIENT DECLINED TO ANSWER,Unknown,1
